# MÓDULO 8: INTRODUCCIÓN A LANGGRAPH
## SESIÓN 2: Diseño de Flujos y Multi-Agentes

**Duración:** 3.5 horas  
**Fecha:** 10/02/2026

---

## Recordatorio de la Sesión 1

En la sesión anterior construimos grafos **lineales**: un nodo conectado al siguiente en secuencia, como una cadena de montaje.

**Hoy vamos más allá:**
- Grafos que **vuelven atrás** (ciclos) para corregir errores
- Grafos que **tienen memoria**
- Grafos que **esperan** a que un humano apruebe antes de continuar

**No estamos programando flujos. Estamos diseñando organizaciones inteligentes.**

Instalamos las dependencias necesarias para trabajar con LangGraph.

In [ ]:
# Instalación de dependencias
!pip install -q langgraph langchain langchain-openai grandalf

Importamos los componentes para construir flujos complejos.

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
import random

---
## BLOQUE 1: Diseño de Flujos de Agentes

### Concepto: De la Línea al Círculo

**En la Sesión 1:** Aprendimos a construir grafos lineales (A → B → C → FIN).

**Hoy:** Aprenderemos a construir grafos **cíclicos** (A → B → volver a A si algo falla).

### El Método de la Pizarra (Whiteboarding)

Antes de escribir código, **dibuja cajas y flechas en papel**.

**Ejercicio mental:**
1. Lista TODAS las tareas que tu agente debe hacer
2. Agrupa las tareas similares (cada grupo = un nodo)
3. Dibuja flechas: ¿Qué pasa primero? ¿Qué depende de qué?
4. Identifica los puntos de decisión: ¿Dónde el agente debe elegir una ruta?

**Esta pizarra ES tu arquitectura.** El código solo la implementa.

---

### 1.1 Ciclos: La Diferencia Entre Impresora y Lápiz con Borrador

**LangChain = Impresora**  
Imprime una vez y termina. No puede volver atrás.

**LangGraph = Lápiz con Borrador**  
Escribe, revisa, borra si está mal, y vuelve a escribir.

**¿Cuándo necesitas ciclos?**
- Generación de código que debe compilar sin errores
- Creación de contenido que debe cumplir criterios de calidad
- Validación de datos que puede requerir múltiples intentos

**Regla de Oro de los Ciclos:**  
Todo ciclo DEBE tener una **condición de salida** clara:
- Contador de intentos máximo (ej: máximo 3 reintentos)
- Flag de éxito (ej: código compila correctamente)
- Timeout (ej: máximo 30 segundos de ejecución)

Sin condición de salida, el agente queda atrapado en un loop infinito.

### Anatomía de un Ciclo

Un ciclo requiere 3 componentes:

1. **Nodo generador:** Produce un output
2. **Nodo validador:** Verifica si el output es aceptable
3. **Router con ciclo:** Decide si continuar o volver al generador

```
[Generar] → [Validar] → [Router]
     ↑                       |
     └───────(si falla)──────┘
                             |
                        (si OK)→ [FIN]
```

### Agente que Genera Números Hasta Encontrar uno Par

In [ ]:
# MEMORIA: Estado con el número generado y contador de intentos
class EstadoGenerador(TypedDict):
    numero: int
    intentos: int
    es_par: bool

# CAPACIDAD 1: Generar número aleatorio
def nodo_generar_numero(state: EstadoGenerador) -> dict:
    """
    CAPACIDAD: Genera un número aleatorio del 1 al 10.
    RESPONSABILIDAD: Solo generar, no validar.
    """
    numero = random.randint(1, 10)
    intentos = state["intentos"] + 1
    
    print(f"Intento {intentos}: Generó el número {numero}")
    
    return {"numero": numero, "intentos": intentos}

# CAPACIDAD 2: Verificar si es par
def nodo_verificar_paridad(state: EstadoGenerador) -> dict:
    """
    CAPACIDAD: Verifica si el número es par.
    RESPONSABILIDAD: Solo verificar, no generar.
    """
    es_par = state["numero"] % 2 == 0
    return {"es_par": es_par}

# DECISIÓN: Router que decide si continuar o terminar
def router_paridad(state: EstadoGenerador) -> str:
    """
    DECISIÓN:
    - Si es par → "terminar" (¡Éxito!)
    - Si intentos >= 5 → "terminar" (Límite alcanzado)
    - Si no → "reintentar" (CICLO: vuelve a generar)
    """
    if state["es_par"]:
        print(f"¡Éxito! Encontré un número par: {state['numero']}")
        return "terminar"
    elif state["intentos"] >= 5:
        print(f"Límite alcanzado. Último número: {state['numero']}")
        return "terminar"
    else:
        print("Es impar, reintentando...")
        return "reintentar"

# CAPACIDAD 3: Nodo final
def nodo_finalizar(state: EstadoGenerador) -> dict:
    return {}

print("Funciones definidas correctamente")

In [ ]:
# ARQUITECTURA: Construir grafo con ciclo
workflow_generador = StateGraph(EstadoGenerador)

# ARQUITECTURA: Registrar capacidades
workflow_generador.add_node("generar", nodo_generar_numero)
workflow_generador.add_node("verificar", nodo_verificar_paridad)
workflow_generador.add_node("finalizar", nodo_finalizar)

# ARQUITECTURA: Edge condicional con CICLO
workflow_generador.add_conditional_edges(
    "verificar",
    router_paridad,
    {
        "reintentar": "generar",  # CICLO: vuelve a generar
        "terminar": "finalizar"   # Salida del ciclo
    }
)

# ARQUITECTURA: Punto de entrada
workflow_generador.set_entry_point("generar")

# ARQUITECTURA: Conexión secuencial simple
workflow_generador.add_edge("generar", "verificar")

# ARQUITECTURA: Conexión final
workflow_generador.add_edge("finalizar", END)

# ARQUITECTURA: Compilar
app_generador = workflow_generador.compile()

print("Grafo con ciclo compilado correctamente")

In [ ]:
# Visualizar el grafo
# print(app_generador.get_graph().draw_ascii()) # Usa LangChain (no acepta ciclos)

from IPython.display import Image, display
# Visualizar el grafo
display(Image(app_generador.get_graph().draw_mermaid_png()))

In [ ]:
# Ejecutar el agente
estado_inicial = {"numero": 0, "intentos": 0, "es_par": False}
resultado = app_generador.invoke(estado_inicial)

print(f"\nResultado final:")
print(f"Número: {resultado['numero']}")
print(f"Intentos totales: {resultado['intentos']}")
print(f"Es par: {resultado['es_par']}")

---
### Micro-reto 1: Añadir Condición de Seguridad al Router

**Objetivo:** Modificar el router para agregar una condición de seguridad adicional.

**Instrucciones:**
1. Modifica `router_paridad_mejorado` para que si el número generado es **mayor que 8**, también termine (sin importar si es par o impar)
2. El router debe devolver `"terminar_especial"` en este caso
3. Conecta `"terminar_especial"` al nodo `nodo_finalizar`

**Pista:** Añade la condición ANTES de verificar si es par.

In [ ]:
# TODO: Modifica el router para añadir la condición de seguridad

def router_paridad_mejorado(state: EstadoGenerador) -> str:
    # TODO: Si número generado es mayor que 8, devolver "terminar_especial"
    
    # Verificamos si es par (código original)
    if state["es_par"]:
        return "terminar"
    elif state["intentos"] >= 5:
        return "terminar"
    else:
        return "reintentar"

# TODO: Reconstruye el grafo usando router_paridad_mejorado
# TODO: Añade la ruta "terminar_especial": "finalizar" en add_conditional_edges

# Descomenta para probar:
app_generador_mejorado = workflow_generador_mejorado.compile()

# Ejecutar el agente
estado_inicial = {"numero": 0, "intentos": 0, "es_par": False}
resultado = app_generador_mejorado.invoke(estado_inicial)

print(f"\nResultado final:")
print(f"Número: {resultado['numero']}")
print(f"Intentos totales: {resultado['intentos']}")
print(f"Es par: {resultado['es_par']}")

from IPython.display import Image, display
# Visualizar el grafo
display(Image(app_generador_mejorado.get_graph().draw_mermaid_png()))

---
### 1.2 Persistencia: Guardar Partida Como en los Videojuegos

**Problema sin persistencia:**  
Si tu agente falla en el paso 5 de un proceso de 10 pasos, debe empezar de nuevo desde el paso 1.

**Solución: Checkpointing (Guardar Partida)**

Como en los videojuegos, puedes **guardar el estado** del agente en cualquier punto. Si algo falla, retomas desde el último checkpoint.

**Componentes de Persistencia:**
- **MemorySaver:** El "disco duro" donde se guardan las partidas
- **thread_id:** El "número de partida" (ej: "partida_usuario_123")
- **Checkpoints automáticos:** LangGraph guarda después de cada nodo

**Casos de uso:**
- Workflows largos que pueden fallar (procesamiento de documentos)
- Conversaciones con usuarios que duran días (chatbot persistente)
- Procesos que esperan aprobación humana (human-in-the-loop)

### Cómo Funciona la Persistencia

Cuando compilas un grafo con `checkpointer=MemorySaver()`:

1. **Después de cada nodo:** El estado se guarda automáticamente
2. **thread_id identifica la partida:** `{"configurable": {"thread_id": "001"}}`
3. **Si falla:** Puedes retomar desde el último checkpoint
4. **Múltiples partidas:** Cada thread_id es independiente

### Contador Persistente

In [ ]:
# MEMORIA: Estado simple
class EstadoContador(TypedDict):
    contador: int
    mensaje: str

# CAPACIDAD: Incrementar contador
def nodo_incrementar(state: EstadoContador) -> dict:
    # state.get es seguro por si viene vacío, asume 0
    contador = state.get("contador", 0) + 1
    mensaje = f"Contador incrementado a {contador}"
    print(mensaje)
    return {"contador": contador, "mensaje": mensaje}

# ARQUITECTURA: Grafo con persistencia
workflow_persistente = StateGraph(EstadoContador)
workflow_persistente.add_node("incrementar", nodo_incrementar)
workflow_persistente.set_entry_point("incrementar")
workflow_persistente.add_edge("incrementar", END)

# ---------------------------------------------------------
# ESCENARIO A: MÉTODO MANUAL (Sin memoria persistente)
# ---------------------------------------------------------
print("=== A: MÉTODO MANUAL (Sin memoria persistente) ===\n")
# Compilamos SIN checkpointer
app_manual = workflow_persistente.compile()

# Paso 1: Tú inicias
resultado1 = app_manual.invoke({"contador": 0, "mensaje": ""})
print(f"Contador: {resultado1['contador']}\n")

# Paso 2: TÚ tienes que tomar el resultado anterior e inyectarlo de nuevo
# Si aquí pusieras {}, el contador se reiniciaría o fallaría
resultado2 = app_manual.invoke({"contador": resultado1['contador'], "mensaje": ""})
print(f"Contador: {resultado2['contador']}\n")

# Paso 3: Lo mismo otra vez
resultado3 = app_manual.invoke({"contador": resultado2['contador'], "mensaje": ""})
print(f"Contador: {resultado3['contador']}\n")


# ---------------------------------------------------------
# ESCENARIO B: MÉTODO AUTOMÁTICO (Con MemorySaver)
# ---------------------------------------------------------
print("\n=== B: MÉTODO AUTOMÁTICO (El grafo recuerda solo) ===\n")

# Compilamos CON checkpointer
memory = MemorySaver()
app_auto = workflow_persistente.compile(checkpointer=memory)

# Configuración: Número de partida
config = {"configurable": {"thread_id": "partida1"}}

# Paso 1: Inicialización (Solo aquí damos el valor inicial)
resultado1 = app_auto.invoke({}, config=config)
print(f"Contador: {resultado1['contador']}\n")

# Paso 2: MAGIA. Pasamos input VACÍO.
# El grafo va a buscar a su memoria el último estado de "partida1"
resultado2 = app_auto.invoke({}, config=config)
print(f"Contador: {resultado2['contador']}\n")

# Paso 3: Input VACÍO de nuevo.
resultado3 = app_auto.invoke({}, config=config)
print(f"Contador: {resultado3['contador']}")

---
### Micro-reto 2: Verificar Independencia de Partidas

**Objetivo:** Comprobar que cada `thread_id` es una partida independiente.

**Instrucciones:**
1. Ejecuta el agente con un nuevo `thread_id` diferente (ej: "partida_002")
2. Inicia el contador desde 10 (en lugar de 0)
3. Realiza 2 invocaciones con este nuevo thread_id
4. Verifica que el contador de "partida_002" NO se mezcla con "partida_001"

**Resultado esperado:**  
- partida_001 debe seguir en 3
- partida_002 debe estar en 12 (10 + 2 incrementos)

In [ ]:

# TODO: Crea una nueva config con thread_id="partida_002"

# TODO: Ejecuta 2 veces con contador inicial = 10

# TODO: Imprime el resultado final de partida_002

# TODO: Verifica que partida_001 sigue intacta ejecutando:
resultado_old = app_auto.invoke({}, config=config)
print(f"Partida 001 - Contador: {resultado_old['contador']} (debe ser 5)")

---
## BLOQUE 2: Sistemas Multi-Agente

### Concepto: El Equipo de Trabajo

Hasta ahora hemos construido **agentes individuales**. Pero los problemas complejos requieren **equipos especializados**.

**Piensa en un equipo de marketing:**
- **Copywriter:** Escribe el contenido
- **Diseñador:** Crea los visuales
- **SEO Specialist:** Optimiza para buscadores
- **Project Manager:** Coordina a todos

**En LangGraph:**
- Cada especialista = Un nodo con capacidades específicas
- El Project Manager = Un router que decide quién trabaja cuándo
- El flujo de trabajo = Las conexiones entre nodos

**Esto es Diseño Organizacional aplicado a agentes.**

### 2.1 Arquitectura Supervisor: El Jefe de Proyecto

**Patrón más común:** Un agente supervisor que delega tareas a especialistas.

**Flujo típico:**
1. Usuario hace una solicitud
2. Supervisor analiza qué se necesita
3. Supervisor delega a uno o varios especialistas
4. Especialistas ejecutan sus tareas
5. Supervisor consolida resultados
6. Supervisor devuelve respuesta al usuario

### Patrón Supervisor Multi-Agente

En un sistema multi-agente:

1. **Nodos especialistas:** Cada uno tiene una responsabilidad única
2. **Estado compartido:** Todos los nodos leen/escriben en el mismo Estado
3. **Router supervisor:** Decide qué especialista trabaja en cada momento
4. **Ciclo de mejora:** El supervisor puede enviar el trabajo de vuelta al especialista si no cumple criterios

### Sistema de Revisión de Contenido

Vamos a construir un sistema con:
- **Escritor:** Genera contenido inicial
- **Revisor:** Verifica calidad del contenido
- **Supervisor:** Decide si aprobar o solicitar correcciones

In [ ]:
# MEMORIA: Estado compartido entre agentes
class EstadoContenido(TypedDict):
    tema: str
    contenido: str
    calidad_ok: bool
    intentos: int
    aprobado: bool

# AGENTE 1: Escritor
def nodo_escritor(state: EstadoContenido) -> dict:
    """
    ESPECIALISTA: Escritor
    RESPONSABILIDAD: Generar contenido sobre el tema
    """
    tema = state["tema"]
    intentos = state["intentos"] + 1
    
    # Simulación: El escritor mejora con cada intento
    if intentos == 1:
        contenido = f"Artículo básico sobre {tema}."
    elif intentos == 2:
        contenido = f"Artículo mejorado sobre {tema} con más detalles."
    else:
        contenido = f"Artículo completo y detallado sobre {tema} con ejemplos."
    
    print(f"Escritor (Intento {intentos}): {contenido}")
    
    return {"contenido": contenido, "intentos": intentos}

# AGENTE 2: Revisor
def nodo_revisor(state: EstadoContenido) -> dict:
    """
    ESPECIALISTA: Revisor
    RESPONSABILIDAD: Verificar calidad del contenido
    """
    contenido = state["contenido"]
    
    # Criterio de calidad: El contenido debe tener más de 50 caracteres
    calidad_ok = len(contenido) > 50
    
    print(f"Revisor: Contenido {'aprobado' if calidad_ok else 'rechazado'} (longitud: {len(contenido)})")
    
    return {"calidad_ok": calidad_ok}

print("Agentes especializados definidos")

In [ ]:
# DECISIÓN: Supervisor (Router)
def router_supervisor(state: EstadoContenido) -> str:
    """
    SUPERVISOR: Project Manager
    RESPONSABILIDAD: Decidir el flujo según calidad y límites
    """
    calidad_ok = state["calidad_ok"]
    intentos = state["intentos"]
    
    
    # DECISIÓN 1: Si la calidad es buena → aprobar
    if calidad_ok:
        print("Supervisor: Contenido aprobado, finalizando.")
        return "traducir"
    
    # DECISIÓN 2: Si llegamos al límite → aprobar de todas formas
    if intentos >= 3:
        print("Supervisor: Límite de intentos alcanzado, aprobando.")
        return "traducir"
    
    # DECISIÓN 3: Si no → solicitar correcciones (CICLO)
    print("Supervisor: Calidad insuficiente, solicitando correcciones.")
    return "corregir"


# NODO: Aprobación final
def nodo_aprobar(state: EstadoContenido) -> dict:
    """
    FINALIZACIÓN: Marca el contenido como aprobado
    """
    print("Sistema: Contenido aprobado y publicado.")
    return {"aprobado": True}

print("Supervisor y nodo de aprobación definidos")

In [ ]:
# ARQUITECTURA: Construir sistema multi-agente
workflow_contenido = StateGraph(EstadoContenido)

# ARQUITECTURA: Registrar especialistas
workflow_contenido.add_node("escritor", nodo_escritor)
workflow_contenido.add_node("revisor", nodo_revisor)
workflow_contenido.add_node("aprobar", nodo_aprobar)

# ARQUITECTURA: Punto de entrada (comienza con el escritor)
workflow_contenido.set_entry_point("escritor")

# ARQUITECTURA: Flujo fijo (escritor → revisor)
workflow_contenido.add_edge("escritor", "revisor")

# ARQUITECTURA: Supervisor decide después del revisor
workflow_contenido.add_conditional_edges(
    "revisor",           # Desde revisor...
    router_supervisor,   # ...el supervisor decide...
    {
        "corregir": "escritor",  # CICLO: vuelve al escritor
        "aprobar": "aprobar"      # Aprueba y finaliza
    }
)

# ARQUITECTURA: Conexión final
workflow_contenido.add_edge("aprobar", END)

# ARQUITECTURA: Compilar
app_contenido = workflow_contenido.compile()

print("Sistema multi-agente compilado")

In [ ]:
# Visualizar el grafo multi-agente
print(app_contenido.get_graph().draw_ascii())

In [ ]:
# Ejecutar sistema multi-agente
estado_inicial = {
    "tema": "Inteligencia Artificial",
    "contenido": "",
    "calidad_ok": False,
    "intentos": 0,
    "aprobado": False
}

resultado = app_contenido.invoke(estado_inicial)

print(f"\n{'='*60}")
print("RESULTADO FINAL:")
print(f"{'='*60}")
print(f"Contenido: {resultado['contenido']}")
print(f"Intentos totales: {resultado['intentos']}")
print(f"Aprobado: {resultado['aprobado']}")

---
### Micro-reto 3: Añadir un Tercer Especialista (Traductor)

**Objetivo:** Expandir el equipo agregando un nodo "Traductor" que se ejecute DESPUÉS de la aprobación.

**Instrucciones:**
1. Crea un nodo `nodo_traductor` que:
   - Reciba el contenido aprobado
   - Lo "traduzca" (simulación: añade el prefijo "[EN] " al contenido)
   - Actualice el campo `contenido` con la versión traducida
2. Agrega el nodo al grafo
3. Cambia la conexión: `aprobar` debe ir a `traductor` (no a END)
4. Conecta `traductor` a END

**Resultado esperado:**  
El contenido final debe tener el prefijo `[EN]` al inicio.

In [ ]:
# TODO: Define el nodo traductor
def nodo_traductor(state: EstadoContenido) -> dict:
    # TODO: Simula traducción añadiendo "[EN] " al inicio del contenido
    return{"contenido": f"[EN] {state['contenido']}"}

def router_supervisor_traduccion(state: EstadoContenido) -> str:
    """
    SUPERVISOR: Project Manager
    RESPONSABILIDAD: Decidir el flujo según calidad y límites
    """
    
    if "[EN]" in state['contenido']:
        return "aprobar"
    return "traducir"


workflow_contenido = StateGraph(EstadoContenido)

# ARQUITECTURA: Registrar especialistas
workflow_contenido.add_node("escritor", nodo_escritor)
workflow_contenido.add_node("revisor", nodo_revisor)
workflow_contenido.add_node("aprobar", nodo_aprobar)

workflow_contenido.add_node("traducir", nodo_traductor)

# ARQUITECTURA: Punto de entrada (comienza con el escritor)
workflow_contenido.set_entry_point("escritor")

# ARQUITECTURA: Flujo fijo (escritor → revisor)
workflow_contenido.add_edge("escritor", "revisor")

# ARQUITECTURA: Supervisor decide después del revisor
workflow_contenido.add_conditional_edges(
    "revisor",           # Desde revisor...
    router_supervisor,   # ...el supervisor decide...
    {
        "traducir": "traducir",  
        "corregir": "escritor"      
    }
)

workflow_contenido.add_conditional_edges(
    "traducir",           
    router_supervisor_traduccion,  
    {
        "traducir": "traducir",  # CICLO: vuelve al escritor
        "aprobar": "aprobar"      # Aprueba y finaliza
    }
)

# ARQUITECTURA: Conexión final
workflow_contenido.add_edge("aprobar", END)

# TODO: Reconstruye el grafo multi-agente
# TODO: Añadir el nodo traductor
# TODO: Configurar conexiones (aprobar → traductor → END)
# TODO: Compilar y ejecutar)

print(f"\n{'='*60}")
print("RESULTADO FINAL:")
print(f"{'='*60}")
print(f"Contenido: {resultado['contenido']}")
print(f"Intentos totales: {resultado['intentos']}")
print(f"Aprobado: {resultado['aprobado']}")


---
### 2.2 Human-in-the-Loop: El Botón de Pausa

**Concepto: El Visto Bueno del Jefe**

Imagina que eres el jefe de un equipo. Tus empleados pueden trabajar de forma autónoma, pero para decisiones importantes quieres **revisar y aprobar** antes de que continúen.

**En agentes:**
- El agente trabaja normalmente
- Antes de un paso crítico, se **pausa** automáticamente
- Espera tu aprobación (o modificaciones)
- Continúa solo cuando tú le das luz verde

**Casos de uso:**
- Aprobar cambios en base de datos
- Revisar emails antes de enviar
- Validar decisiones financieras
- Confirmar acciones irreversibles

### Cómo Funciona interrupt_before

Al compilar un grafo puedes especificar:

```python
app = workflow.compile(
    checkpointer=MemorySaver(),
    interrupt_before=["nodo_critico"]
)
```

**Comportamiento:**
1. El grafo se ejecuta normalmente
2. Al llegar a `nodo_critico`, se **pausa** (no lo ejecuta)
3. Devuelve el control al código
4. El humano puede:
   - Aprobar: `app.invoke(None, config=config)`
   - Modificar estado: `app.update_state(config, {"campo": nuevo_valor})`
   - Cancelar: No invocar de nuevo

### Sistema de Aprobación de Pedidos

In [ ]:
# MEMORIA: Estado de pedido
class EstadoPedido(TypedDict):
    producto: str
    cantidad: int
    precio_total: float
    confirmado: bool

# CAPACIDAD 1: Calcular precio
def nodo_calcular_precio(state: EstadoPedido) -> dict:
    # Precio unitario fijo: 10 euros
    precio_total = state["cantidad"] * 10.0
    print(f"Sistema: Precio calculado: {precio_total} EUR")
    return {"precio_total": precio_total}

# CAPACIDAD 2: Confirmar pedido (PUNTO DE PAUSA)
def nodo_confirmar_pedido(state: EstadoPedido) -> dict:
    print(f"Sistema: Pedido confirmado para {state['producto']}")
    return {"confirmado": True}

# ARQUITECTURA: Grafo con human-in-the-loop
workflow_pedido = StateGraph(EstadoPedido)

workflow_pedido.add_node("calcular_precio", nodo_calcular_precio)
workflow_pedido.add_node("confirmar_pedido", nodo_confirmar_pedido)

workflow_pedido.set_entry_point("calcular_precio")
workflow_pedido.add_edge("calcular_precio", "confirmar_pedido")
workflow_pedido.add_edge("confirmar_pedido", END)

# HUMAN-IN-THE-LOOP: Pausar ANTES de confirmar el pedido
app_pedido = workflow_pedido.compile(
    checkpointer=MemorySaver(),
    interrupt_before=["confirmar_pedido"]  # PAUSA antes de este nodo
)

print("Grafo con human-in-the-loop compilado")

In [ ]:
# Configuración con thread_id para persistencia
config_pedido = {"configurable": {"thread_id": "pedido_001"}}

# FASE 1: Ejecutar hasta el punto de pausa
print("=== FASE 1: Cálculo automático ===")
estado_inicial = {
    "producto": "Laptop",
    "cantidad": 3,
    "precio_total": 0.0,
    "confirmado": False
}

resultado_pausa = app_pedido.invoke(estado_inicial, config=config_pedido)

print(f"\nEstado después de cálculo:")
print(f"Producto: {resultado_pausa['producto']}")
print(f"Cantidad: {resultado_pausa['cantidad']}")
print(f"Precio total: {resultado_pausa['precio_total']} EUR")
print(f"Confirmado: {resultado_pausa['confirmado']}")
print("\n⏸ SISTEMA PAUSADO - Esperando aprobación humana...")
print("\n(En producción, aquí esperarías input del usuario)")

# Sin modificaciones
# resultado_pausa = app_pedido.invoke(None, config=config_pedido)

# Cambiamos la cantidad
# app_pedido.update_state(config=config_pedido, values={"cantidad":4})
# print(app_pedido.get_state)

estado_modificado = app_pedido.update_state(config=config_pedido, values={"cantidad": 4})
resultado_pausa_modificar = app_pedido.invoke(estado_modificado, config=config_pedido)

print(f"Confirmado: {resultado_pausa_modificar['confirmado']}")

resultado_pausa = app_pedido.invoke(None, config=config_pedido)

print(f"Producto: {resultado_pausa['producto']}")
print(f"Cantidad: {resultado_pausa['cantidad']}")
print(f"Precio total: {resultado_pausa['precio_total']} EUR")
print(f"Confirmado: {resultado_pausa['confirmado']}")

**Nota sobre Human-in-the-Loop:**

En un entorno real:
1. El sistema se pausaría aquí
2. El humano revisaría el pedido (precio, cantidad)
3. El humano podría:
   - Aprobar: `app_pedido.invoke(None, config=config_pedido)`
   - Modificar: `app_pedido.update_state(config_pedido, {"cantidad": 2})`
   - Cancelar: No continuar la ejecución

El uso de `thread_id` + `checkpointer` permite que el sistema recuerde exactamente dónde se pausó.

---
### Micro-reto 4: Cambiar el Punto de Interrupción

**Objetivo:** Modificar el grafo para que se pause en un nodo diferente.

**Instrucciones:**
1. Crea un nuevo nodo `nodo_enviar_email` que simule enviar un email de confirmación
2. Inserta este nodo DESPUÉS de `confirmar_pedido` y ANTES de END
3. Cambia `interrupt_before` para que pause antes de `enviar_email` (no antes de `confirmar_pedido`)
4. Ejecuta el grafo y verifica que:
   - El pedido se confirma automáticamente
   - El sistema se pausa ANTES de enviar el email

**Resultado esperado:**  
El output debe mostrar "Pedido confirmado" pero NO "Email enviado".

In [5]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# MEMORIA: Estado de pedido
class EstadoPedido(TypedDict):
    producto: str
    cantidad: int
    precio_total: float
    confirmado: bool
    email: str
    email_enviado: bool

# CAPACIDAD 1: Calcular precio
def nodo_calcular_precio(state: EstadoPedido) -> dict:
    # Precio unitario fijo: 10 euros
    precio_total = state["cantidad"] * 10.0
    print(f"Sistema: Precio calculado: {precio_total} EUR")
    return {"precio_total": precio_total}

# CAPACIDAD 2: Confirmar pedido (PUNTO DE PAUSA)
def nodo_confirmar_pedido(state: EstadoPedido) -> dict:
    print(f"Sistema: Pedido confirmado para {state['producto']}")
    return {"confirmado": True}

def nodo_enviar_email(state: EstadoPedido) -> dict:
    if state["email"]:
        return{"email_enviado": True}
    else:
        return{"email_enviado": False}

# ARQUITECTURA: Grafo con human-in-the-loop
workflow_pedido = StateGraph(EstadoPedido)

workflow_pedido.add_node("calcular_precio", nodo_calcular_precio)
workflow_pedido.add_node("confirmar_pedido", nodo_confirmar_pedido)
workflow_pedido.add_node("enviar_email", nodo_enviar_email)

workflow_pedido.set_entry_point("calcular_precio")
workflow_pedido.add_edge("calcular_precio", "confirmar_pedido")
workflow_pedido.add_edge("confirmar_pedido", "enviar_email")
workflow_pedido.add_edge("enviar_email", END)

# HUMAN-IN-THE-LOOP: Pausar ANTES de confirmar el pedido
app_pedido = workflow_pedido.compile(
    checkpointer=MemorySaver(),
    interrupt_before=["confirmar_pedido", "enviar_email"] 
)

print("Grafo con human-in-the-loop compilado")

# Configuración con thread_id para persistencia
config_pedido = {"configurable": {"thread_id": "pedido_002"}}


# FASE 1: Ejecutar hasta el punto de pausa
print("=== FASE 1: Cálculo automático ===")
estado_inicial = {
    "producto": "Laptop",
    "cantidad": 3,
    "precio_total": 0.0,
    "confirmado": False,
    "email": "",
    "email_enviado": False
}


resultado_pausa = app_pedido.invoke(estado_inicial, config=config_pedido)


print(f"\nEstado después de cálculo:")
print(f"Producto: {resultado_pausa['producto']}")
print(f"Cantidad: {resultado_pausa['cantidad']}")
print(f"Precio total: {resultado_pausa['precio_total']} EUR")
print(f"Confirmado: {resultado_pausa['confirmado']}")
respuesta = input("\nescriba S si aprueba o N si no")
print("\n(En producción, aquí esperarías input del usuario)")


# Sin modificaciones
resultado_pausa
if respuesta.strip().lower() == "s":
    resultado_pausa = app_pedido.invoke(None, config=config_pedido)
elif respuesta.strip().lower() == "n":
    modificar = input("Desea modificar algo? (S/N)")
    if modificar.strip().lower() == "s":
        cantidad = int(input("Indique la nueva cantidad"))
        resultado_pausa = app_pedido.update_state(config=config_pedido, values={"cantidad": cantidad})
        resultado_pausa = app_pedido.invoke(resultado_pausa, config=config_pedido)
        resultado_pausa = app_pedido.invoke(None, config=config_pedido)
        
email_user = input("\nIndique su email: ")

resultado_pausa = app_pedido.update_state(config=config_pedido, values={"email": email_user})
resultado_pausa = app_pedido.invoke(resultado_pausa, config=config_pedido)
resultado_pausa = app_pedido.invoke(None, config=config_pedido)




print(f"\nProducto: {resultado_pausa['producto']}")
print(f"Cantidad: {resultado_pausa['cantidad']}")
print(f"Precio total: {resultado_pausa['precio_total']} EUR")
print(f"Confirmado: {resultado_pausa['confirmado']}")
print(f"Enviado a email: {resultado_pausa['email']}")


Grafo con human-in-the-loop compilado
=== FASE 1: Cálculo automático ===
Sistema: Precio calculado: 30.0 EUR

Estado después de cálculo:
Producto: Laptop
Cantidad: 3
Precio total: 30.0 EUR
Confirmado: False



(En producción, aquí esperarías input del usuario)
Sistema: Pedido confirmado para Laptop
Sistema: Precio calculado: 30.0 EUR
Sistema: Pedido confirmado para Laptop

Producto: Laptop
Cantidad: 3
Precio total: 30.0 EUR
Confirmado: True
Enviado a email: mail
